# <b>Start here</b> - Initial environment setup
Ensure to run each step in order - Optional steps may be skipped if desired

In [ ]:
#@title Check which Nvidia GPU has been assigned to you
!nvidia-smi

In [ ]:
#@title Step 1 - Set up the environment (Mandatory)
onGoogleColab = True
onWindows = False
previouslySetup = False
GPUID = 0

import os

installPath = '/content'
privilegeEscalation = "sudo"

if onWindows:
    onGoogleColab = False
    installPath = 'C:\\Users\\Heylon\\Documents\\RIFEstuff\\RIFE-Colab'

if onGoogleColab == False:
    privilegeEscalation = "pkexec"
    if not onWindows:
        installPath = '/home/heylon/RIFE-Colab'

pathSeparator = os.path.sep
print(pathSeparator)

os.chdir(installPath)


# Download from google drive --------------------------
import requests
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
# -------------------------------------------------------

if not onWindows and not previouslySetup:
    !"{privilegeEscalation}" add-apt-repository 'deb http://au.archive.ubuntu.com/ubuntu/ focal main restricted'
    !"{privilegeEscalation}" add-apt-repository 'deb http://au.archive.ubuntu.com/ubuntu/ focal universe'
    #!"{privilegeEscalation}" add-apt-repository -y ppa:jonathonf/ffmpeg-4
    !"{privilegeEscalation}" apt update
    !"{privilegeEscalation}" apt -y install ffmpeg

!git clone https://github.com/HeylonNHP/RIFE-Colab.git RIFEcolab

In [ ]:
#@title Step 2 - Run to setup RIFE (Mandatory)
os.chdir(installPath + '/RIFEcolab')
from rifeFunctions import *
downloadRIFE(installPath,onWindows)
os.chdir(installPath)
from RIFEcolab.rifeInterpolationFunctions import *
setupRIFE(installPath,GPUID)

In [ ]:
#@title Step 3 - Run to initialise interpolation procedures (Mandatory)
import subprocess
import re
import math 
import shutil
import math
os.chdir(installPath)
from RIFEcolab.frameChooser import chooseFrames
from RIFEcolab.FFmpegFunctions import *

def extractFrames(inputFile,projectFolder,mode,mpdecimateSensitivity="64*12,64*8,0.33"):
    '''
    Equivalent to DAINAPP Step 1
    '''
    os.chdir(projectFolder)
    if os.path.exists("original_frames"):
        shutil.rmtree("original_frames")
    if not os.path.exists("original_frames"):
        os.mkdir("original_frames")
    
    if mode == 1:
        !ffmpeg -i "{inputFile}" -map_metadata -1 -pix_fmt rgb24 "original_frames"/%15d.png
    elif mode == 3 or mode == 4:
        hi, lo, frac = mpdecimateSensitivity.split(",")
        mpdecimate = "mpdecimate=hi={}:lo={}:frac={}".format(hi, lo, frac)
        !ffmpeg -i "{inputFile}" -map_metadata -1 -pix_fmt rgb24 -copyts -r 1000 -vsync 0 -frame_pts true -vf "{mpdecimate}" -qscale:v 1 "original_frames"/%15d.png

def runInterpolator(inputFile, projectFolder, interpolationFactor, loopable, mode, scenechangeSensitivity):
    '''
    Equivalent to DAINAPP Step 2
    '''
    os.chdir(installPath+'/arXiv2020-RIFE/')
    origFramesFolder = projectFolder + '/' + "original_frames"
    interpFramesFolder = projectFolder + '/' + "interpolated_frames"

    if not os.path.exists(interpFramesFolder):
        os.mkdir(interpFramesFolder)

    # Get output FPS
    outputFPS = 0
    if mode == 1 or mode == 3:
        outputFPS = getFPS(inputFile) * interpolationFactor
    elif mode == 4:
        outputFPS = (getFrameCount(inputFile,True) / getLength(inputFile)) * interpolationFactor

    # Loopable
    if loopable:
        generateLoopContinuityFrame(origFramesFolder)
    
    files = os.listdir(origFramesFolder)
    files.sort()

    if mode == 1:
        
        count = 0
        shutil.copy(origFramesFolder + '/' + files[0],interpFramesFolder + '/' + '{:015d}.png'.format(count))

        for i in range(0,len(files)-1):
            print("Interpolating frame:", i+1, "Of", len(files), "{:.2f}%".format(((i+1)/len(files))*100))
            shutil.copy(origFramesFolder + '/' + files[i],interpFramesFolder + '/' + '{:015d}.png'.format(count))
            shutil.copy(origFramesFolder + '/' + files[i+1],interpFramesFolder + '/' + '{:015d}.png'.format(count+interpolationFactor))

            currentFactor = interpolationFactor

            while currentFactor > 1:
                period = int(interpolationFactor/currentFactor)
                for j in range(0,period):
                    offset = int(currentFactor * j)
                    beginFrame = interpFramesFolder + '/' + '{:015d}.png'.format(count+offset)
                    endFrame = interpFramesFolder + '/' + '{:015d}.png'.format(count+currentFactor+offset)
                    middleFrame = interpFramesFolder + '/' + '{:015d}.png'.format(count+int(currentFactor/2)+offset)
                    rifeInterpolate(beginFrame,endFrame,middleFrame,scenechangeSensitivity)
                currentFactor = int(currentFactor/2)

            count += interpolationFactor
      
    elif mode == 3 or mode == 4:
        count = 0
        shutil.copy(origFramesFolder + '/' + files[0],interpFramesFolder + '/' + '{:015d}.png'.format(count))

        for i in range(0,len(files)-1):
            
            localInterpolationFactor = interpolationFactor
            # If mode 3, calculate interpolation factor on a per-frame basis to maintain desired FPS
            
            beginFrameTime = int(files[i][:-4])
            endFrameTime = int(files[i+1][:-4])
            timeDiff = endFrameTime - beginFrameTime
            if mode == 3:
                localInterpolationFactor = 2

                while 1/(((endFrameTime-beginFrameTime)/localInterpolationFactor)/1000) < outputFPS:
                    localInterpolationFactor = int(localInterpolationFactor * 2)

            print("Interpolating frame:", i+1, "Of", len(files), "{:.2f}%".format(((i+1)/len(files))*100), "Frame interp. factor", "{}x".format(localInterpolationFactor))

            # Get timecodes of both working frames
            currentTimecode = int(files[i][:-4])
            nextTimecode = int(files[i+1][:-4])
            
            shutil.copy(origFramesFolder + '/' + files[i],interpFramesFolder + '/' + '{:015d}.png'.format(currentTimecode))
            shutil.copy(origFramesFolder + '/' + files[i+1],interpFramesFolder + '/' + '{:015d}.png'.format(nextTimecode))

            currentFactor = localInterpolationFactor

            while currentFactor > 1:
                period = int(round(localInterpolationFactor/currentFactor))
                for j in range(0,period):
                    # Get offset from the first frame
                    offset = (timeDiff/period) * j
                    #print(currentTimecode,offset, "j ",j)
                    # Time difference between 'begin' and 'end' frames relative to current interpolation factor (Block size)
                    currentFactor2 = (currentFactor/localInterpolationFactor) * timeDiff
                    beginFrameTime = int(currentTimecode+offset)
                    endFrameTime = int(currentTimecode+currentFactor2+offset)
                    middleFrameTime = int(currentTimecode+(currentFactor2/2)+offset)

                    beginFrame = interpFramesFolder + '/' + '{:015d}.png'.format(beginFrameTime)
                    endFrame = interpFramesFolder + '/' + '{:015d}.png'.format(endFrameTime)
                    middleFrame = interpFramesFolder + '/' + '{:015d}.png'.format(middleFrameTime)

                    #print("times",beginFrameTime,middleFrameTime,endFrameTime)
                    rifeInterpolate(beginFrame,endFrame,middleFrame,scenechangeSensitivity)
                currentFactor = int(currentFactor/2)

            # count += interpolationFactor
    # Loopable
    if loopable:
        removeLoopContinuityFrame(interpFramesFolder)

    return [outputFPS]

def generateLoopContinuityFrame(framesFolder):
    files = os.listdir(framesFolder)
    files.sort()

    # Find the distance between the first two and last two frames
    # Use this distance to calculate position for the loop continuity frame
    beginFirst = int(files[0][:-4])
    endFirst = int(files[1][:-4])

    beginLast = int(files[-2][:-4])
    endLast = int(files[-1][:-4])

    averageDistance = int(((endFirst-beginFirst) + (endLast-beginLast)) / 2)

    # Create frame
    shutil.copy(framesFolder + '/' + files[0], framesFolder + '/' + '{:015d}.png'.format(endLast + averageDistance))
    print("Made loop continuity frame:",framesFolder + '/' + '{:015d}.png'.format(endLast + averageDistance))

def removeLoopContinuityFrame(framesFolder):
    files = os.listdir(framesFolder)
    files.sort()
    os.remove(framesFolder + '/' + files[-1])

def createOutput(inputFile, projectFolder, outputVideo, outputFPS, loopable, mode, crfout, useNvenc):
    '''
    Equivalent to DAINAPP Step 3
    '''
    os.chdir(projectFolder)
    maxLoopLength = 15
    preferredLoopLength = 10
    inputLength = getLength(inputFile)

    inputFFmpeg = ""

    encoderPreset = "-pix_fmt yuv420p -c:v libx264 -preset veryslow -crf {}".format(crfout)

    if useNvenc:
        encoderPreset = "-pix_fmt yuv420p -c:v h264_nvenc -gpu {} -preset p7 -profile high -cq {}".format(GPUID,crfout+10)

    if mode == 1:
        inputFFmpeg = "-r " + str(outputFPS) + " -i interpolated_frames/%15d.png"
    if mode == 3 or mode == 4:
        #generateTimecodesFile(projectFolder)
        chooseFrames(projectFolder + pathSeparator + "interpolated_frames", outputFPS)
        inputFFmpeg = "-vsync 1 -r {} -f concat -safe 0 -i interpolated_frames/framesCFR.txt".format(outputFPS)

    if loopable == False or (maxLoopLength / float(inputLength) < 2):
        # Don't loop, too long input
        print('Dont loop',maxLoopLength / float(inputLength))

        !ffmpeg -hide_banner -stats -loglevel error -y {inputFFmpeg} -i "{inputFile}" -map 0 -map 1:a? -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" {encoderPreset} "{outputVideo}"
    else:
        loopCount = math.ceil(preferredLoopLength / float(inputLength)) - 1
        loopCount = str(loopCount)
        print('Loop', loopCount)
        
        !ffmpeg -y -stream_loop {loopCount} -i "{inputFile}" -vn loop.flac
        
        audioInput = ""
        if os.path.exists('loop.flac'):
            audioInput = "-i loop.flac -map 0 -map 1"
            print("Looped audio exists")
        !ffmpeg -hide_banner -stats -loglevel error -y -stream_loop {loopCount} {inputFFmpeg} {audioInput} -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" {encoderPreset} "{outputVideo}"
        if os.path.exists('loop.flac'):
            os.remove('loop.flac')

def generateTimecodesFile(projectFolder):
    '''
    Scan a folder full of PNG frames and generate a timecodes file for FFmpeg
    '''
    os.chdir(projectFolder)
    files = os.listdir("interpolated_frames")
    files.sort()

    # Generate duration for last frame
    beginFirst = int(files[0][:-4])
    endFirst = int(files[1][:-4])

    beginLast = int(files[-2][:-4])
    endLast = int(files[-1][:-4])

    averageDistance = int(((endFirst-beginFirst) + (endLast-beginLast)) / 2)

    f = open("interpolated_frames/frames.txt", "w")
    for i in range(0,len(files)-1):
        # Calculate time duration between frames
        currentFrameFile = files[i]
        nextFrameFile = files[i+1]
        # Each file has a .png extention, use [:-4] to remove
        frameDuration = int(int(nextFrameFile[:-4]) - int(currentFrameFile[:-4]))
        # Write line
        f.write("file '" + projectFolder + "/interpolated_frames/" + currentFrameFile +"'\nduration "+ "{:.5f}".format(float(frameDuration/1000.0)) + "\n")
    
    # Generate last frame
    lastFrameName = "{:015d}.png".format(endLast)
    f.write("file '" + projectFolder + "/interpolated_frames/" + lastFrameName +"'\nduration "+ "{:.5f}".format(float(averageDistance/1000.0)) + "\n")
    f.close()

def performAllSteps(inputFile,interpolationFactor,loopable,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity, useNvenc):
    
    projectFolder = inputFile[:inputFile.rindex(pathSeparator)]
    if nonLocalPNGs:
        projectFolder = installPath + pathSeparator + "tempFrames"
        if not os.path.exists(projectFolder):
            os.mkdir(projectFolder)
    
    # Clear pngs if they exist
    if os.path.exists(projectFolder + '/' + 'original_frames'):
        shutil.rmtree(projectFolder + '/' + 'original_frames')

    if os.path.exists(projectFolder + '/' + 'interpolated_frames'):
        shutil.rmtree(projectFolder + '/' + 'interpolated_frames')

    extractFrames(inputFile,projectFolder,mode,mpdecimateSensitivity)

    outParams = runInterpolator(inputFile, projectFolder, interpolationFactor, loopable, mode, scenechangeSensitivity)
    print('---INTERPOLATION DONE---')
    outputVideoName = inputFile[:inputFile.rindex(pathSeparator)+1]
    outputVideoName += '{:.2f}fps-{}x-mode{}-rife-output.mp4'.format(outParams[0],interpolationFactor,mode)

    createOutput(inputFile, projectFolder, outputVideoName, outParams[0], loopable, mode, crf, useNvenc)

    if clearPNGs:
        shutil.rmtree(projectFolder + '/' + 'original_frames')
        shutil.rmtree(projectFolder + '/' + 'interpolated_frames')

In [ ]:
#@title Step 4 - Mount your google drive to use as storage for input/output videos (Optional)
from google.colab import drive
drive.mount('/content/drive')

# Processing a single video (Extract -> Interpolate -> Create output - All at once)
**Processing a single video has now been broken into two steps:**<br>
Run the **set settings for interpolation** step first to apply the desired settings for interpolation, then run the second step to start the interpolator. <br><br>
Paste in the path to your video, choose your desired settings, then run this code box to interpolate a single video
<br>
The output will be in the same folder as the input
<br><br>
<<< - Click the folder icon on the left hand side to open the file browser<br>
Navigate to the file you'd like to interpolate<br>
Right click it, click Copy Path and paste it over the top of the example path below

In [ ]:
#@title Run to set settings for interpolation (DO FIRST)
#@markdown Input video path
inputFile = '/content/drive/MyDrive/Videos/original_long.mp4' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode =  3#@param{type:"number"}
#@markdown Enable perfect loop (For GIFs or looping videos)
loopable = False #@param{type:"boolean"}
#@markdown Interpolation factor (2, 4, 8, 16, 32... etc)
interpolationFactor = 2 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.20 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}
#@markdown Use NVENC for output (Broken on Colab)
useNvenc = False #@param{type:"boolean"}

In [ ]:
#@title Interpolate a single video - Run interpolator (DO SECOND)
performAllSteps(inputFile,interpolationFactor,loopable,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity, useNvenc)

# Processing a single video (Run steps individually)
Please run **Run to set settings for interpolation (DO FIRST)** first, before running the steps below in order **Extract frames -> Run interpolator -> Create output**
<br><br>
Running the steps individually like this will allow you to extract the frames, check for duplicate frames that were missed by FFmpeg after extraction and manually delete them, then run the interpolator, and check for any missed scene changes in the interpolated frames before creating the output

In [ ]:
#@title Extract frames (step 1)
projectFolder = inputFile[:inputFile.rindex(pathSeparator)]
if nonLocalPNGs:
    projectFolder = installPath + pathSeparator + "tempFrames"
    if not os.path.exists(projectFolder):
        os.mkdir(projectFolder)

# Clear pngs if they exist
if os.path.exists(projectFolder + '/' + 'original_frames'):
    shutil.rmtree(projectFolder + '/' + 'original_frames')

if os.path.exists(projectFolder + '/' + 'interpolated_frames'):
    shutil.rmtree(projectFolder + '/' + 'interpolated_frames')

extractFrames(inputFile,projectFolder,mode,mpdecimateSensitivity)

In [ ]:
#@title Run interpolator (step 2)
outParams = runInterpolator(inputFile, projectFolder, interpolationFactor, loopable, mode, scenechangeSensitivity)
print('---INTERPOLATION DONE---')

In [ ]:
#@title Create output (step 3)
outputVideoName = inputFile[:inputFile.rindex(pathSeparator)+1]
outputVideoName += '{:.2f}fps-{}x-mode{}-rife-output.mp4'.format(outParams[0],interpolationFactor,mode)

createOutput(inputFile, projectFolder, outputVideoName, outParams[0], loopable, mode, crf, useNvenc)

if clearPNGs:
    shutil.rmtree(projectFolder + '/' + 'original_frames')
    shutil.rmtree(projectFolder + '/' + 'interpolated_frames')

# Processing multiple videos using the batch interpolator
set inputDirectory to equal the folder path containing the input videos
<br><br>
Each video must be in its own subdirectory within the input folder
<br><br>
The interpolation factor for each video is chosen based on its input FPS (Accounting for duplicate frames if using mode 4). It will choose an interpolation factor which will meet or exceed the 'Output FPS target'
<br><br>
For perfectly looping GIFs/videos: Add [l] to the filename or subfolder name of the input file. E.G.:<br>
inputLoopingGifFileNumberOne[l].gif
<br>
or
<br>
InputLoopingGifFolder[l]/inputLoopingGifFileNumberOne.gif

In [ ]:
#@title Interpolate a folder of videos
#@markdown Input folder path
inputDirectory = '/content' #@param{type:"string"}
#@markdown Frame handling mode (Available choices: 1, 3, or 4. Equivalent to DAINAPP)
mode = 3 #@param{type:"number"}
#@markdown Output FPS target
fpsTarget = 59 #@param{type:"number"}
#@markdown x264 CRF value for producing the output
crf = 20 #@param{type:"number"}
#@markdown Clear the original_frames and interpolated frames after completion
clearPNGs = True #@param{type:"boolean"}
#@markdown Don't store the PNG files in the same folder as the input video
nonLocalPNGs = True #@param{type:"boolean"}
#@markdown Scenechange sensitivity (lower is more sensitive)
scenechangeSensitivity = 0.20 #@param{type:"number"}
#@markdown Sensitivity of mpdecimate when detecting and removing duplicate frames [hi,lo,frac] (mode 3/4)
mpdecimateSensitivity = "64*12,64*8,0.33" #@param{type:"string"}
#@markdown Use NVENC for output (Broken on Colab)
useNvenc = False #@param{type:"boolean"}

print(torch.__version__) 

import os
import traceback

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(inputDirectory):
    for file in f:
      files.append(os.path.join(r, file))

files.sort()

for inputVideoFile in files:
    try:
      print(inputVideoFile)

      if mode == 1 or mode == 3:
        currentFPS = getFPS(inputVideoFile)
      elif mode == 4 or mode == 3:
        currentFPS = getFrameCount(inputVideoFile,True) / getLength(inputVideoFile)

      # Attempt to interpolate everything to above 59fps
      targetFPS = fpsTarget
      exponent = 1
      if currentFPS < targetFPS:
        while (currentFPS * (2 ** exponent)) < targetFPS:
          exponent += 1
      else:
        continue
      # use [l] to denote whether the file is a loopable video
      print("looping?", '[l]' in inputVideoFile)
      if '[l]' in inputVideoFile:
        print("LOOP")
        performAllSteps(inputVideoFile,(2 ** exponent),True,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity,useNvenc)
      else:
        print("DON'T LOOP")
        performAllSteps(inputVideoFile,(2 ** exponent),False,mode,crf,clearPNGs,nonLocalPNGs,scenechangeSensitivity,mpdecimateSensitivity,useNvenc)
    except:
      traceback.print_exc()
